In [ ]:
epochs = Config['N_EPOCHS']

for epoch_i in tqdm(range(epochs), desc=f"{epochs} epochs", position=0, leave=True):
    # training set
    train_loss, train_accuracy = [], []
    iter_n = len(train_dataset)
    
    with tqdm(total=iter_n, desc=f"{iter_n} iterations", leave=False) as progress_bar:
        for _batch in train_dataset:
            batch=_batch[0]  # train_dataset is tuple containing (image,labels)
            labels=_batch[1]

            batch = jnp.array(batch, dtype=jnp.float32)
            labels = jnp.array(labels, dtype=jnp.float32)
            
            batch, labels = shard(batch), shard(labels)
        
            # backprop and update param & batch statsp
            
            state, train_metadata, dropout_rng = parallel_train_step(state, batch, labels, dropout_rng)
            train_metadata = unreplicate(train_metadata)
            
            # update train statistics
            _train_loss, _train_top1_acc = map(float, [train_metadata['loss'], *train_metadata['accuracy']])
            train_loss.append(_train_loss)
            train_accuracy.append(_train_top1_acc)
            progress_bar.update(1)
            
    avg_train_loss = sum(train_loss)/len(train_loss)
    avg_train_acc = sum(train_accuracy)/len(train_accuracy)
    print(f"[{epoch_i+1}/{Config['N_EPOCHS']}] Train Loss: {avg_train_loss:.03} | Train Accuracy: {avg_train_acc:.03}")
    
    # validation set
    
    valid_accuracy = []
    iter_n = len(test_dataset)
    with tqdm(total=iter_n, desc=f"{iter_n} iterations", leave=False) as progress_bar:
        for _batch in test_dataset:
            batch = _batch[0]
            labels = _batch[1]

            batch = jnp.array(batch, dtype=jnp.float32)
            labels = jnp.array(labels, dtype=jnp.float32)

            batch, labels = shard(batch), shard(labels)
            metric = parallel_val_step(state, batch, labels)[0]
            valid_accuracy.append(metric)
            progress_bar.update(1)


    avg_valid_acc = sum(valid_accuracy)/len(valid_accuracy)
    avg_valid_acc = np.array(avg_valid_acc)[0]
    print(f"[{epoch_i+1}/{Config['N_EPOCHS']}] Valid Accuracy: {avg_valid_acc:.03}")